In [1]:
# import all packages
 
import numpy as np
import time

In [2]:
def load_data():
    # load data
    data = np.loadtxt('DMDdata200.csv', delimiter=',')
    x1 = data[0:,1:-1]
    x2 = data[0:,2:]
    return x1, x2	

In [3]:
x1, x2 = load_data()

In [4]:
data_shape = (x1.shape[1], x1.shape[0])
percent_Trunc = 0.90
dt = 2.5e-05

In [5]:
# Compression matrix generation

C_gaussian = np.random.normal(0, 1, size=data_shape)

In [6]:
# compress data

y1 = np.dot(C_gaussian, x1)
y2 = np.dot(C_gaussian, x2)

In [7]:
# cDMD

def DMD(y1, y2, x1, x2, percent_Trunc, dt):

    [Uc, Sc, Vc] = np.linalg.svd(y1, full_matrices=False, compute_uv=True, hermitian=False)

    menergyc = 0.0
    for i in range(0, np.size(Sc)):
        menergyc += Sc[i]/np.sum(Sc)
        if menergyc > percent_Trunc:
            break
        rc = i
        
    menergyc, rc

    Atildec = Uc[:, :rc].T @ y2 @ Vc[:, :rc] * np.reciprocal(Sc[: rc])
    [eivValc,eivVecc] = np.linalg.eig(Atildec)

    phic = x2 @ Vc[:rc, :].T @ np.diag(np.reciprocal(Sc[: rc])) @ eivVecc
    bc = np.linalg.lstsq(phic, x1[:, 0])

    num_modesc = phic.shape[1]
    omegac = np.log(eivValc) / dt
            
    return phic, bc, omegac, num_modesc

In [8]:
## Reconstruction of the data

def recon(phi, b, omega, num_steps, r):
    time_dynamics = np.zeros((r, num_steps), dtype=complex)
    for i in range(num_steps):
        time_dynamics[:, i] = b[0] * np.exp(omega * i)
    return phi @ time_dynamics

In [9]:
startc = 0
endc = 0

startc = time.time()

phic, bc, omegac, num_modesc = DMD(y1, y2, x1, x2, percent_Trunc, dt)

endc = time.time()

print("Time taken for cDMD: ", endc - startc)

start = 0
end = 0

start = time.time()

phi, b, omega, num_modes = DMD(x1, x2, x1, x2, percent_Trunc, dt)

end = time.time()

print("Time taken for DMD: ", end - start)

C:\Users\sumuk\AppData\Local\Temp\ipykernel_1892\2249330943.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  bc = np.linalg.lstsq(phic, x1[:, 0])
C:\Users\sumuk\AppData\Local\Temp\ipykernel_1892\2249330943.py:23: RuntimeWarning: invalid value encountered in log
  omegac = np.log(eivValc) / dt


Time taken for cDMD:  5.298781871795654
Time taken for DMD:  37.9315927028656


In [11]:
reconstructed_fieldc = recon(phic, bc, omegac, 2000, num_modesc)
reconstructed_field = recon(phi, b, omega, 2000, num_modes)